In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../src/')
import dataset_prep

Import de la base de données

In [18]:
selected_columns = [
"nb_packets_sent",
"distance",
"distRealSR1",
"pos_y_rec_f",
"pos_y_rec",
"pos_x_rec_f",
"pos_x_rec",
"pos_x_send",
"pos_y_send",
"spd_x_send",
"spd_y_send",
"sendTime",
"receiver",
"sender",
"label"
]

In [42]:
df = dataset_prep.import_dataset('../data/DoS_0709_new_columns.csv', selected_columns)

In [43]:
dataset_prep.clean_dataset(df)

Nombre de lignes avant nettoyage :  4753433
Nombre de lignes après nettoyage :  4709293


Analyse et vérification des données

In [44]:
senders = np.unique(df["sender"].values)
senders

array([    9,    15,    21, ..., 24399, 24405, 24411], dtype=int64)

In [45]:
print(len(senders))

4067


In [46]:
df.loc[df['sender'] == senders[0]].sort_values("sendTime")

,sendTime,sender,receiver,pos_x_send,pos_y_send,spd_x_send,spd_y_send,label,pos_x_rec,pos_y_rec,pos_x_rec_f,pos_y_rec_f,nb_packets_sent,distRealSR1,distance
4,25212.602877,9,45,257.604242,141.917136,-2.409908,14.316189,0,265.540271,46.207434,268.539855,49.801087,96,7.936029,96.038156
5,25213.602877,9,45,255.242943,156.206787,-2.566640,14.210027,0,268.539855,49.801087,268.050977,55.331302,96,13.296912,107.233302
6,25214.602878,9,45,252.857751,170.495740,-2.342829,14.254332,0,268.050977,55.331302,270.113953,63.318293,96,15.193227,116.162309
7,25215.602879,9,45,250.525544,184.832952,-2.352874,14.315002,0,270.113953,63.318293,268.836896,72.915064,96,19.588409,123.083379
8,25216.602880,9,45,248.511212,199.166902,-2.079440,14.360857,0,268.836896,72.915064,267.267270,84.617270,96,20.325683,127.877519
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,25291.602880,9,225,134.821280,1024.506204,-0.602227,8.137469,0,138.085770,1102.386398,138.801402,1096.647173,96,3.264490,77.948582
65,25291.602880,9,147,134.821280,1024.506204,-0.602227,8.137469,0,147.593919,898.797866,147.021568,894.975244,96,12.772639,126.355556
91,25292.602869,9,219,134.539820,1030.197090,-0.270067,3.653434,0,142.457918,1043.790636,143.221108,1033.675922,96,7.918098,15.731521
95,25292.602869,9,225,134.539820,1030.197090,-0.270067,3.653434,0,138.801402,1096.647173,139.551488,1089.075886,96,4.261582,66.586594


In [48]:
# Copie de la bdd pour récuperer les colonnes sans impacter la vraie base de données

data_copy = df.copy()
data_copy = data_copy.drop(["label"], axis=1)

columns = data_copy.columns

Amélioration à faire :

Séléctionner seulement les sender avec value_counts > 20

Séparation de la base de données : diviser pour régner

In [ ]:
data_copy.sort_values("sender")[:50]

In [84]:
# /!\ attention à bien utiliser la base de données rangée par sender pour limiter la perte de données
# Indice égal à 0, 1 ou 2
# Célia = 0
# Chléo = 1
# Thomas = 2

def split_data_in_3(df, indice):
    # Division de la bdd en 3
    split_data = np.array_split(df,3)

    # Division en plusieurs sous-bdd pour diminuer le temps de calcul par exécution
    #divided_data = np.array_split(split_data[indice],50)

    return split_data

/!\ à bien changer l'indice !!

In [85]:
divided_dataframes = split_data_in_3(df.sort_values("sender"),0)

In [86]:
len(divided_dataframes)

3

Création des séquences

In [87]:
def sequence_creation(df):
    sequence_array = []
    label_array = []

    for sender in senders:
        length = df.loc[df['receiver'] == sender].shape[0]
        slide = 10
        start = 0
        end = 20
        # On vérifie qu'il est possible de faire une séquence de taille 20
        while length > 20:
            # Extraction par tranche de 20 avec une inter de 10
            sequence = (df.loc[df['sender'] == sender].sort_values("sendTime"))[start:end]

            # Labels correspondant
            labels = sequence["label"]

            # Autres colonnes
            sequence = sequence.drop(["label"], axis=1)

            # Attribution des tableaux numpy
            sequence_array.append(np.asarray(sequence))
            label_array.append(labels)

            # Mise à jour des variables
            start += slide
            end += slide
            length -= 20
    
    return sequence_array, label_array

/!\ à bien changer l'indice !!

In [88]:
sequence_test, label_test = sequence_creation(divided_dataframes[0])

Vérification des séquences obtenues

In [90]:
len(sequence_test)

77580

In [89]:
sequence_1 = pd.DataFrame(sequence_test[0])
sequence_1.columns=columns
sequence_1

,sendTime,sender,receiver,pos_x_send,pos_y_send,spd_x_send,spd_y_send,pos_x_rec,pos_y_rec,pos_x_rec_f,pos_y_rec_f,nb_packets_sent,distRealSR1,distance
0,25212.602877,9.0,45.0,257.604242,141.917136,-2.409908,14.316189,265.540271,46.207434,268.539855,49.801087,96.0,7.936029,96.038156
1,25213.602877,9.0,45.0,255.242943,156.206787,-2.566640,14.210027,268.539855,49.801087,268.050977,55.331302,96.0,13.296912,107.233302
2,25214.602878,9.0,45.0,252.857751,170.495740,-2.342829,14.254332,268.050977,55.331302,270.113953,63.318293,96.0,15.193227,116.162309
3,25215.602879,9.0,45.0,250.525544,184.832952,-2.352874,14.315002,270.113953,63.318293,268.836896,72.915064,96.0,19.588409,123.083379
4,25216.602880,9.0,45.0,248.511212,199.166902,-2.079440,14.360857,268.836896,72.915064,267.267270,84.617270,96.0,20.325683,127.877519
5,25217.602881,9.0,45.0,246.612236,213.485932,-2.108686,14.351979,267.267270,84.617270,264.934535,98.201431,96.0,20.655034,130.513458
6,25218.602882,9.0,45.0,244.608441,227.849499,-2.116586,14.373925,264.934535,98.201431,262.838049,112.387093,96.0,20.326094,131.231748
7,25219.602870,9.0,45.0,242.408756,242.184475,-2.155661,14.337441,262.838049,112.387093,260.602987,126.570242,96.0,20.429293,131.395268
8,25220.602870,9.0,45.0,240.160195,256.522426,-2.156116,14.340466,260.602987,126.570242,258.087496,140.783737,96.0,20.442792,131.550286
9,25221.602871,9.0,45.0,237.925559,270.860484,-2.153866,14.325500,258.087496,140.783737,255.565275,154.966493,96.0,20.161937,131.630027
